In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import scanpy as sc
from flexiznam.config import PARAMETERS
from pathlib import Path
import iss_preprocess as iss
from iss_preprocess.pipeline.segment import segment_spots
from iss_preprocess.pipeline.segment import make_cell_dataframe
from itertools import cycle
import brainglobe_atlasapi as bga
import editdistance
import tifffile
import ast
from umi_tools import UMIClusterer

data_path = "becalia_rabies_barseq/BRYC65.1d/chamber_13/"
processed_path = Path(PARAMETERS["data_root"]["processed"])
metadata = iss.io.load_metadata(data_path)
becalia_path = "/nemo/lab/znamenskiyp/home/users/becalia/data/BRYC65.1d/"
ops = iss.io.load_ops(data_path)

In [ ]:
redo = False
# For each roi take all cell masks and remove those outside of the cerebrum
if redo:

    def convert_array_values(arr, value_list):
        bool_arr = np.full(arr.shape, False, dtype=bool)
        indices = np.isin(arr, value_list)
        bool_arr[indices] = True
        return bool_arr

    bga.config.write_config_value(
        key="brainglobe_dir",
        val="/nemo/lab/znamenskiyp/home/shared/resources/.brainglobe/",
    )
    atlas_name = "allen_mouse_%dum" % 10
    bg_atlas = bga.bg_atlas.BrainGlobeAtlas(
        atlas_name,
        "/nemo/lab/znamenskiyp/home/shared/resources/.brainglobe/",
        check_latest=False,
    )
    cerebrum_acronyms = bg_atlas.get_structure_descendants("CH")
    fiber_acronyms = bg_atlas.get_structure_descendants("cc") + [
        "cing",
        "dhc",
        "alv",
        "scwm",
        "or",
        "ar",
        "amc",
        "ee",
        "ec",
    ]
    cerebrum_acronyms = cerebrum_acronyms + fiber_acronyms
    cerebrum_ids = bg_atlas._get_from_structure(cerebrum_acronyms, "id") + [
        0
    ]  # Zero is outside, include L1

    for roi in [1, 2, 5, 6]:
        if roi == 1:
            masks = np.load(becalia_path + f"shifted_masks_roi_{roi}.npy")

        else:
            masks = np.load(processed_path / data_path / f"masks_{roi}.npy")

        area_image = iss.pipeline.make_area_image(
            data_path=data_path, roi=roi, atlas_size=10, full_scale=False
        )
        cerebrum_mask = convert_array_values(area_image, cerebrum_ids)

        # Scale cerebrum mask to cells mask and pad to account for rounding errors
        expanded_bool_arr = np.repeat(np.repeat(cerebrum_mask, 10, axis=0), 10, axis=1)
        pad_width = (
            (0, masks.shape[0] - expanded_bool_arr.shape[0]),
            (0, masks.shape[1] - expanded_bool_arr.shape[1]),
        )
        padded_bool_arr = np.pad(
            expanded_bool_arr, pad_width, mode="constant", constant_values=False
        )

        # Set cell masks to zero outside of cerebrum
        masks[~padded_bool_arr] = 0
        print(f"Saving masks and spots for roi {roi}")
        if roi == 1:
            np.save(
                f"/nemo/lab/znamenskiyp/home/users/becalia/data/BRYC65.1d/shifted_masks_cerebrum_roi_{roi}.npy",
                masks,
            )
        else:
            np.save(
                f"/nemo/lab/znamenskiyp/home/users/becalia/data/BRYC65.1d/masks_cerebrum_roi_{roi}.npy",
                masks,
            )

In [ ]:
# Generate expanded masks - takes some time
if redo:
    from skimage.segmentation import expand_labels

    pixel_size = 0.18

    for roi in [1, 2, 5, 6]:
        if roi == 1:
            masks = np.load(becalia_path + f"shifted_masks_cerebrum_roi_{roi}.npy")
            big_mask = expand_labels(masks, distance=int(5 / pixel_size))
            np.save(becalia_path + f"big_mask_shifted_roi_{roi}_cerebrum.npy", big_mask)
        else:
            masks = np.load(becalia_path + f"masks_cerebrum_roi_{roi}.npy")
            big_mask = expand_labels(masks, distance=int(5 / pixel_size))
            np.save(becalia_path + f"big_mask_roi_{roi}_cerebrum.npy", big_mask)

In [ ]:
# Thresholds
spot_score_threshold = 0.1
hyb_score_threshold = 0.8
# Barcodes to be filtered later with harsher thresholds
barcode_dot_threshold = 0.1
barcode_num_th = 5


# Load barcode spots and gene spots within expanded masks
for roi in [1, 2, 5, 6]:
    if redo:
        if roi == 1:
            # Load
            big_mask = np.load(
                becalia_path + f"big_mask_shifted_roi_{roi}_cerebrum.npy"
            )
        else:
            big_mask = np.load(becalia_path + f"big_mask_roi_{roi}_cerebrum.npy")

        def reorder_labels(label_image):
            """
            rearranges the labels so that they are a sequence of integers
            """

            _, idx = np.unique(label_image.flatten(), return_inverse=True)
            return idx.reshape(label_image.shape).astype("uint32")

        # Reordering mask labels is necessary for running pciSeq
        # As we have removed some masks, we need to reorder them to be a sequence of integers
        big_mask = reorder_labels(big_mask)
        np.save(becalia_path + f"reordered_big_mask_roi_{roi}.npy", big_mask)

    else:
        big_mask = np.load(becalia_path + f"reordered_big_mask_roi_{roi}.npy")

    # Find which barcode is in which cells
    spot_dict = iss.pipeline.segment.add_mask_id(
        data_path,
        roi,
        mask_expansion=None,
        masks=big_mask,
        barcode_dot_threshold=barcode_dot_threshold,
        spot_score_threshold=spot_score_threshold,
        hyb_score_threshold=hyb_score_threshold,
    )
    barcode_spots = spot_dict["barcode_round"]
    barcode_in_cell = barcode_spots[barcode_spots.mask_id != 0]
    barcode_in_cell = barcode_in_cell[
        barcode_in_cell.dot_product_score > barcode_dot_threshold
    ]
    barcode_in_cell["roi"] = roi

    # Concatenate all cells with barcodes, forcing each roi to have unique mask_ids
    if roi == 1:
        print(f"Making first cell df roi {roi}")
        cell_df = make_cell_dataframe(
            data_path, roi, masks=big_mask, mask_expansion=None, atlas_size=10
        )
        cell_df["roi"] = roi

        all_roi_barcodes = barcode_in_cell.copy()

        barcode_df, genes_df = segment_spots(
            data_path,
            roi=roi,
            mask_expansion=None,
            masks=big_mask,
            barcode_dot_threshold=barcode_dot_threshold,
            spot_score_threshold=spot_score_threshold,
            hyb_score_threshold=hyb_score_threshold,
        )
        max_mask_id = genes_df.index.max()

        genes_df["roi"] = roi

        barcoded_cells = barcode_df[barcode_df.sum(axis=1) > barcode_num_th].iloc[1:]

        code_len = len(barcoded_cells.columns[0])
        distance_df = pd.DataFrame(
            index=barcoded_cells.index, columns=np.arange(code_len + 1), dtype=int
        )
        for cell_id, cell in barcoded_cells.iterrows():
            main = cell.idxmax()
            dst = np.zeros(code_len + 1)
            barcodes = cell[cell != 0]
            for seq, cnt in barcodes.items():
                edit = editdistance.eval(seq, main)
                dst[edit] += cnt
            distance_df.loc[cell_id, :] = dst

    else:
        print(f"Making cell df roi {roi}")
        cell_df_next = make_cell_dataframe(
            data_path, roi, masks=big_mask, mask_expansion=None, atlas_size=10
        )
        cell_df_next.index = cell_df_next.index + max_mask_id
        cell_df_next["roi"] = roi
        cell_df = pd.concat([cell_df, cell_df_next])

        barcode_in_cell["mask_id"] = barcode_in_cell["mask_id"] + max_mask_id
        all_roi_barcodes = pd.concat([all_roi_barcodes, barcode_in_cell])

        barcode_df, genes_df_next = segment_spots(
            data_path,
            roi=roi,
            mask_expansion=None,
            masks=big_mask,
            barcode_dot_threshold=barcode_dot_threshold,
            spot_score_threshold=spot_score_threshold,
            hyb_score_threshold=hyb_score_threshold,
        )
        genes_df_next["roi"] = roi
        genes_df_next = genes_df_next[1:]
        genes_df_next.index = genes_df_next.index + max_mask_id
        genes_df = pd.concat([genes_df, genes_df_next])

        barcoded_cells = barcode_df[barcode_df.sum(axis=1) > barcode_num_th].iloc[1:]

        code_len = len(barcoded_cells.columns[0])
        distance_df_next = pd.DataFrame(
            index=barcoded_cells.index + max_mask_id,
            columns=np.arange(code_len + 1),
            dtype=int,
        )
        for cell_id, cell in barcoded_cells.iterrows():
            main = cell.idxmax()
            dst = np.zeros(code_len + 1)
            barcodes = cell[cell != 0]
            for seq, cnt in barcodes.items():
                edit = editdistance.eval(seq, main)
                dst[edit] += cnt
            distance_df_next.loc[cell_id, :] = dst
        distance_df = pd.concat([distance_df, distance_df_next])

        max_mask_id = genes_df.index.max()

In [ ]:
# Output
if False:
    # cell_df
    cell_df.to_pickle(becalia_path + "roi1_2_5_6_cell_df.pkl")  # not yet saved
    # genes_df
    genes_df.to_pickle(becalia_path + "roi1_2_5_6_genes_df.pkl")
    # distance_df
    distance_df.to_pickle(becalia_path + "roi1_2_5_6_distance_df.pkl")  # not yet saved
    # all_roi_barcodes
    all_roi_barcodes.to_pickle(becalia_path + "all_roi_barcodes.pkl")

In [ ]:
# Load pciSeq gene/cell data and select cells with/without barcodes

# Select threshold for RV pos cells
barcode_count = 10
barcode_dot_product_threshold = 0.2

for roi in [1, 2, 5, 6]:
    if roi == 1:
        pciseq_data = pd.read_csv(
            f"/nemo/lab/znamenskiyp/home/users/becalia/data/pciSeq/BRYC65.1d/20230608_ROI{roi}_shifted_cerebrum_yao2019_whole_cortex_allcells_subclass_Slc17a7/cellData.tsv",
            delimiter="\t",
        )
        pciseq_data["roi"] = roi
    else:
        pciseq_data_next = pd.read_csv(
            f"/nemo/lab/znamenskiyp/home/users/becalia/data/pciSeq/BRYC65.1d/20230608_ROI{roi}_shifted_cerebrum_yao2019_whole_cortex_allcells_subclass_Slc17a7/cellData.tsv",
            delimiter="\t",
        )
        pciseq_data_next["roi"] = roi
        pciseq_data = pd.concat([pciseq_data, pciseq_data_next], ignore_index=True)

    pciseq_data["Cell_Num"] = pciseq_data.index + 1

roi_pass_thresh = all_roi_barcodes[
    (all_roi_barcodes["dot_product_score"] > barcode_dot_product_threshold)
]
counts = roi_pass_thresh["mask_id"].value_counts()
high_abundance_cells = counts[counts >= barcode_count]

pciseq_rv_pos = pciseq_data[pciseq_data["Cell_Num"].isin(high_abundance_cells.index)]

plt.title("pciSeq RV+ cells")
plt.scatter(pciseq_rv_pos["X"].values, pciseq_rv_pos["Y"].values, s=0.1)
plt.gca().invert_yaxis()
plt.gca().set_aspect("equal")

pd.options.mode.chained_assignment = None  # default='warn'

pciseq_rv_pos["Genenames"] = pciseq_rv_pos["Genenames"].apply(
    lambda s: list(ast.literal_eval(s))
)
pciseq_rv_pos["CellGeneCount"] = pciseq_rv_pos["CellGeneCount"].apply(
    lambda s: list(ast.literal_eval(s))
)
pciseq_rv_pos["ClassName"] = pciseq_rv_pos["ClassName"].apply(
    lambda s: list(ast.literal_eval(s))
)
pciseq_rv_pos["Prob"] = pciseq_rv_pos["Prob"].apply(lambda s: list(ast.literal_eval(s)))
pciseq_rv_pos["BestClass"] = pciseq_rv_pos.apply(
    lambda row: row["ClassName"][pd.Series(row["Prob"]).idxmax()], axis=1
)
pciseq_rv_pos["BestProb"] = pciseq_rv_pos.apply(lambda r: np.max(r["Prob"]), axis=1)


unique_genes = pciseq_rv_pos["Genenames"].explode().unique()

# create new dataframe with cells as rows and genes as columns
cell_gene_matrix_df = pd.DataFrame(columns=unique_genes)

for i, row in pciseq_rv_pos.iterrows():
    for gene, count in zip(row["Genenames"], row["CellGeneCount"]):
        cell_gene_matrix_df.at[i, gene] = count

cell_gene_matrix_df.fillna(0, inplace=True)
cell_gene_matrix_df.index = pciseq_rv_pos.Cell_Num.values


# Select subsample of non RV infected cells
pciseq_subsample = pciseq_data[
    ~pciseq_data["Cell_Num"].isin(high_abundance_cells.index)
].sample(20000)
plt.title("pciSeq RV- cells")
plt.scatter(pciseq_subsample["X"].values, pciseq_subsample["Y"].values, s=0.1)
plt.gca().invert_yaxis()
plt.gca().set_aspect("equal")

pciseq_subsample["Genenames"] = pciseq_subsample["Genenames"].apply(
    lambda s: ast.literal_eval(s) if isinstance(s, str) else s
)
pciseq_subsample["CellGeneCount"] = pciseq_subsample["CellGeneCount"].apply(
    lambda s: ast.literal_eval(s) if isinstance(s, str) else s
)
pciseq_subsample["ClassName"] = pciseq_subsample["ClassName"].apply(
    lambda s: ast.literal_eval(s) if isinstance(s, str) else s
)
pciseq_subsample["Prob"] = pciseq_subsample["Prob"].apply(
    lambda s: ast.literal_eval(s) if isinstance(s, str) else s
)
pciseq_subsample["BestClass"] = pciseq_subsample.apply(
    lambda row: row["ClassName"][pd.Series(row["Prob"]).idxmax()]
    if isinstance(row["ClassName"], list)
    and isinstance(row["Prob"], list)
    and row["Prob"]
    else row["BestClass"],
    axis=1,
)
pciseq_subsample["BestProb"] = pciseq_subsample.apply(
    lambda r: np.max(r["Prob"])
    if isinstance(r["Prob"], list) and r["Prob"]
    else r["BestProb"],
    axis=1,
)

unique_genes = pciseq_subsample["Genenames"].explode().unique()

# create new dataframe with cells as rows and genes as columns
neg_cell_gene_matrix_df = pd.DataFrame(
    index=pciseq_subsample.index, columns=unique_genes
)

for i, row in pciseq_subsample.iterrows():
    for gene, count in zip(row["Genenames"], row["CellGeneCount"]):
        neg_cell_gene_matrix_df.at[i, gene] = count

neg_cell_gene_matrix_df.fillna(0, inplace=True)
neg_cell_gene_matrix_df.index = pciseq_subsample["Cell_Num"].values

In [ ]:
# Plot whole section pciSeq cell types


def plot_cell_types(cell_data):
    colors = {
        #        'L2/3': 'deepskyblue',
        #        'L4': 'dodgerblue',
        #        'L5 IT': 'blue',
        #        'L5 NP': 'magenta',
        #        'L5 PT': 'blueviolet',
        #        'L6 CT':  'forestgreen',
        #        'L6 IT': 'violet',
        #        'L6b': 'black',
        #        'Pvalb': 'darkorange',
        #        'Sst':  'orangered',
        #        'Sncg': 'deeppink',
        #        'Serpinf1': 'limegreen',
        #        'Lamp5': 'tomato',
        #        'Vip': 'crimson'
        "Astro": "black",
        "CA1-ProS": "darkorchid",
        "CA2-IG-FC": "sandybrown",
        "CA3": "fuchsia",
        "CR": "teal",
        "CT SUB": "darkolivegreen",
        "Car3": "aquamarine",
        "DG": "hotpink",
        "Endo": "dimgray",
        "L2 IT ENTl": "red",
        "L2 IT ENTm": "coral",
        "L2/3 IT CTX": "brown",
        "L2/3 IT ENTl": "indianred",
        "L2/3 IT PPP": "orangered",
        "L2/3 IT RHP": "darkred",
        "L3 IT ENT": "darkorange",
        "L4 RSP-ACA": "gold",
        "L4/5 IT CTX": "yellow",
        "L5 IT CTX": "greenyellow",
        "L5 PPP": "chartreuse",
        "L5 PT CTX": "palegreen",
        "L5/6 IT TPE-ENT": "forestgreen",
        "L5/6 NP CTX": "darkgreen",
        "L6 CT CTX": "aqua",
        "L6 IT CTX": "deepskyblue",
        "L6 IT ENTl": "lightskyblue",
        "L6b CTX": "darkslateblue",
        "L6b/CT ENT": "midnightblue",
        "Lamp5": "fuchsia",
        "Meis2": "blue",
        "Micro-PVM": "lightpink",
        "NP PPP": "lime",
        "NP SUB": "seagreen",
        "Oligo": "gray",
        "Pvalb": "violet",
        "SMC-Peri": "beige",
        "SUB-ProS": "paleturquoise",
        "Sncg": "orchid",
        "Sst": "blueviolet",
        "Sst Chodl": "hotpink",
        "VLMC": "thistle",
        "Vip": "plum",
        "Zero": "gainsboro",
    }
    markers = cycle("ov^<>spPXD*")
    fig, ax = plt.subplots(figsize=(80, 80))  # plt.figure(figsize=(80,80))

    # Plot downsampled ara ref stitched image
    processed_path = "/camp/lab/znamenskiyp/home/shared/projects/"
    data_path = "becalia_rabies_barseq/BRYC65.1d/chamber_13/"
    image = tifffile.imread(
        processed_path
        + data_path
        + f"register_to_ara/chamber_13_r{roi}_sl00{roi}.ome.tif"
    )
    image[image > 150] = 0
    scale = 10
    fig.show()
    image_artist = ax.imshow(image, cmap="gray_r", vmax=500, origin="lower")
    image_artist.set_extent(np.array(image_artist.get_extent()) * scale)
    ax.autoscale(False)

    zero_class = cell_data[cell_data["BestClass"] == "Zero"]
    cell_data = cell_data[cell_data["BestClass"] != "Zero"]
    ax = plt.subplot(1, 1, 1)
    clusters = np.sort(cell_data["BestClass"].unique())
    for i, cluster in enumerate(clusters):
        color = "gray"
        for type in colors:
            if cluster.find(type) != -1:
                color = colors[type]
        plt.plot(
            cell_data[cell_data["BestClass"] == cluster]["X"],
            cell_data[cell_data["BestClass"] == cluster]["Y"],
            next(markers),
            c=color,
            markersize=6,
            # alpha = 0.6
        )
    plt.legend(clusters, loc="lower center", ncol=2, prop={"size": 45}, markerscale=8)
    # leg = plt.gca().get_legend()
    # for handle in leg.legendHandles:
    #    handle.set_sizes([30])
    # plt.legend(clusters, loc='lower center', ncol=2)
    # , bbox_to_anchor=(0.,0.,1.5,1.)
    plt.plot(zero_class["X"], zero_class["Y"], "o", markersize=2, c="lightgrey")
    ax.set_aspect("equal", "box")
    ax.invert_yaxis()
    plt.tight_layout()
    plt.axis("off")

In [ ]:
plot_cell_types(pciseq_data[pciseq_data["roi"] == 1])

In [ ]:
cell_hierarchy = {
    "Hippocampal": ["CA1-ProS", "CA2-IG-FC", "CA3", "DG", "SUB-ProS"],
    "L2/3": [
        "L2 IT ENTl",
        "L2 IT ENTm",
        "L2/3 IT CTX",
        "L2/3 IT ENTl",
        "L2/3 IT PPP",
        "L2/3 IT RHP",
        "L3 IT ENT",
    ],
    "L4/5": ["L4 RSP-ACA", "L4/5 IT CTX"],
    "L5 IT": ["L5 IT CTX", "L5/6 IT TPE-ENT"],
    "L5 PT": ["L5 PPP", "L5 PT CTX"],
    "L5 NP": ["L5/6 NP CTX", "NP PPP", "NP SUB"],
    "L6 CT": ["L6 CT CTX", "CT SUB"],
    "L6 IT": ["L6 IT CTX", "L6 IT ENTl"],
    "L6b": ["L6b CTX", "L6b/CT ENT"],
    # "GABAergic" : ['Vip', 'Lamp5', 'Sst',
    #  'Sst Chodl', 'Sncg', 'Pvalb'],
    "Vip": ["Vip"],
    "Lamp5": ["Lamp5"],
    "Sst": ["Sst", "Sst Chodl"],
    "Sncg": ["Sncg"],
    "Pvalb": ["Pvalb"],
    "Non-neuronal": ["Astro", "Endo", "Micro-PVM", "Oligo", "SMC-Peri", "VLMC"],
}

In [ ]:
# Make adata object for RV+ cells
adata = sc.AnnData(cell_gene_matrix_df)
adata.obs["BestClass"] = pciseq_rv_pos["BestClass"].values
adata.obs["Barcode_counts"] = high_abundance_cells.sort_index().values
adata.obs["broad_class"] = adata.obs["BestClass"].apply(
    lambda x: next((key for key, value in cell_hierarchy.items() if x in value), None)
)

sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
adata.raw = adata
sc.pp.normalize_total(adata, target_sum=10)
sc.pp.log1p(adata)
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30)
sc.tl.umap(adata, min_dist=0.1)
sc.tl.leiden(adata, resolution=1)


# Make adata_neg object for subsample of RV- cells
adata_neg = sc.AnnData(neg_cell_gene_matrix_df)
adata_neg.obs["BestClass"] = pciseq_subsample["BestClass"].values
adata_neg.obs["broad_class"] = adata_neg.obs["BestClass"].apply(
    lambda x: next((key for key, value in cell_hierarchy.items() if x in value), None)
)

sc.pp.calculate_qc_metrics(adata_neg, percent_top=None, log1p=False, inplace=True)
adata_neg.raw = adata_neg
sc.pp.normalize_total(adata_neg, target_sum=10)
sc.pp.log1p(adata_neg)
sc.tl.pca(adata_neg, svd_solver="arpack")
sc.pp.neighbors(adata_neg, n_neighbors=10, n_pcs=30)
sc.tl.umap(adata_neg, min_dist=0.1)
sc.tl.leiden(adata_neg, resolution=1)

In [ ]:
sc.pl.umap(
    adata_neg,
    ncols=2,
    color=["Slc17a7", "Gad1", "Sst", "broad_class", "BestClass"],
    frameon=False,
    size=400,
)

In [ ]:
sc.pl.umap(
    adata,
    ncols=2,
    color=["Slc17a7", "Gad1", "Sst", "Barcode_counts", "broad_class", "BestClass"],
    frameon=False,
    size=400,
)

In [ ]:
# Plot all genes dotplot for RV+ cells
sc.pl.dotplot(
    adata,
    var_names=adata.var.index,  # ['Slc17a7', 'Pcp4', 'Gad1', 'Vip', 'Sst', 'Npy', 'Pvalb']
    groupby="broad_class",
)

In [ ]:
# Plot selected genes dotplot for RV+ cells
neuron_groups = [
    "L2/3",
    "L4/5",
    "L5 IT",
    "L5 NP",
    "L5 PT",
    "L6 CT",
    "L6 IT",
    "L6b",
    "Vip",
    "Sst",
    "Lamp5",
    "Pvalb",
    "Sncg",
]
neuron_adata = adata[adata.obs["broad_class"].isin(neuron_groups)]

dotplot = sc.pl.dotplot(
    neuron_adata,
    var_names=[
        "Slc17a7",
        "Nrn1",
        "Chgb",
        "Pde1a",
        "Pcp4",
        "Rgs4",
        "Necab1",
        "Cplx1",
        "Cplx3",
        "Enpp2",
        "Gad1",
        "Cnr1",
        "Lamp5",
        "Vip",
        "Sst",
        "Npy",
        "Pvalb",
    ],
    groupby="broad_class",
    show=False,
)

# Modify the x-labels position
plt.xticks(
    np.arange(len(neuron_groups)), neuron_groups, rotation="vertical", ha="center"
)

# Adjust the plot size
fig = plt.gcf()
# fig.set_size_inches(12, 5)

plt.savefig(
    "/nemo/lab/znamenskiyp/home/users/becalia/figs/2023_UCL_sympo/dotplot_pos.svg",
    bbox_inches="tight",
)

In [ ]:
# Plot selected genes dotplot for RV- cells
euron_groups = [
    "L2/3",
    "L4/5",
    "L5 IT",
    "L5 NP",
    "L5 PT",
    "L6 CT",
    "L6 IT",
    "L6b",
    "Vip",
    "Sst",
    "Lamp5",
    "Pvalb",
    "Sncg",
]
neuron_adata = adata_neg[adata_neg.obs["broad_class"].isin(neuron_groups)]

dotplot = sc.pl.dotplot(
    neuron_adata,
    var_names=[
        "Slc17a7",
        "Nrn1",
        "Chgb",
        "Pde1a",
        "Pcp4",
        "Rgs4",
        "Necab1",
        "Cplx1",
        "Cplx3",
        "Enpp2",
        "Gad1",
        "Cnr1",
        "Lamp5",
        "Vip",
        "Sst",
        "Npy",
        "Pvalb",
    ],
    groupby="broad_class",
    show=False,
)

# Modify the x-labels position
plt.xticks(
    np.arange(len(neuron_groups)), neuron_groups, rotation="vertical", ha="center"
)

# Adjust the plot size
fig = plt.gcf()
# fig.set_size_inches(12, 5)

plt.savefig(
    "/nemo/lab/znamenskiyp/home/users/becalia/figs/2023_UCL_sympo/dotplot_neg.svg",
    bbox_inches="tight",
)

In [ ]:
# Plot the proportion of cell types for adata and adata_neg

# Exclude None values from adata_neg
adata_neg_filtered = adata_neg[~adata_neg.obs["broad_class"].isna()]

# Calculate the proportions of cell types for adata_neg
neg_cell_types, neg_counts = np.unique(
    adata_neg_filtered.obs["broad_class"], return_counts=True
)
neg_proportions = neg_counts / np.sum(neg_counts)

# Exclude None values from adata_pos
adata_pos_filtered = adata[~adata.obs["broad_class"].isna()]

# Calculate the proportions of cell types for adata_pos
pos_cell_types, pos_counts = np.unique(
    adata_pos_filtered.obs["broad_class"], return_counts=True
)
pos_proportions = pos_counts / np.sum(pos_counts)

# Get the unique cell types from both datasets
all_cell_types = np.unique(np.concatenate([neg_cell_types, pos_cell_types]))


df_new = pd.DataFrame(
    columns=neg_proportions.index.values, index=["adata_pos", "adata_neg"]
)  # ['adata']+
df_new.loc["adata_pos"] = pos_proportions
df_new.loc["adata_neg"] = neg_proportions
df_new["adata"] = df_new.index

df_new.plot(
    x="adata", kind="barh", stacked=True, title="Stacked Bar Graph", mark_right=True
)

# plt.axis('off')

plt.savefig(
    "/nemo/lab/znamenskiyp/home/users/becalia/figs/2023_UCL_sympo/stacked_bar_celltype.svg",
    bbox_inches="tight",
)

In [ ]:
# Plot barcode spots coloured by dot product scores
fig, axs = plt.subplots(4, 1, figsize=(30, 40))
rois = [1, 2, 5, 6]
filtered_spots = all_roi_barcodes[
    (all_roi_barcodes["roi"] == roi) & (all_roi_barcodes["dot_product_score"] > 0.2)
]

for i, ax in enumerate(axs):
    roi = rois[i]
    scatter = ax.scatter(
        filtered_spots.x,
        filtered_spots.y,
        s=0.01,
        c=filtered_spots["dot_product_score"],
        cmap="viridis",
    )
    ax.set_aspect("equal")
    ax.axis("off")
    ax.set_xlim(2000, 24000)
    ax.set_ylim(3000, 20000)
    ax.invert_yaxis()
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.2, pad=0.02)
    cbar.set_label("Colorbar Label")
plt.tight_layout()

In [ ]:
# Plot sorted dot product scores and spot mean scores
dot_product_scores = all_roi_barcodes["dot_product_score"]
sorted_indices = np.argsort(dot_product_scores)
sorted_spot_scores = all_roi_barcodes["spot_score"].values[sorted_indices]
sorted_mean_scores = all_roi_barcodes["mean_score"].values[sorted_indices]

plt.plot(np.sort(all_roi_barcodes["dot_product_score"]))
plt.scatter(range(len(sorted_mean_scores)), sorted_mean_scores, s=1, alpha=0.3, c="r")
plt.show()

In [ ]:
# Plot sorted dot product scores and spot mean scores
plt.plot(np.sort(all_roi_barcodes["dot_product_score"]))
plt.scatter(range(len(sorted_spot_scores)), sorted_spot_scores, s=3, alpha=0.3, c="r")
plt.show()

In [ ]:
# Collapse in situ barcodes using UMITools
edit_distance_threshold = 2

# Add up counts of each sequence
column_data = all_roi_barcodes[all_roi_barcodes["dot_product_score"] > 0.2]["bases"]
sequence_dict = {}
for sequence in column_data:
    sequence_bytes = sequence.encode()
    if sequence_bytes in sequence_dict:
        sequence_dict[sequence_bytes] += 1
    else:
        sequence_dict[sequence_bytes] = 1
dict_df = pd.DataFrame(
    list(sequence_dict.items()), columns=["sequence", "counts_before_collapse"]
)
dict_df["sequence"] = dict_df["sequence"].apply(lambda x: x.decode())

# Create a UMIClusterer object and perform collapsing
clusterer = UMIClusterer("adjacency")
clustered_umis = clusterer(sequence_dict, threshold=edit_distance_threshold)
clustered_umis

# Extract the first element and length for each inner list
data = [(lst[0].decode(), len(lst)) for lst in clustered_umis]


most_abundant_sequence = []
sums = []
for lst in clustered_umis:
    # Convert bytes to string
    most_abundant_sequence.append(lst[0].decode())
    total_collapsed_counts = 0
    for sequence in lst:
        sequence = sequence.decode()
        total_collapsed_counts += dict_df.loc[dict_df["sequence"] == sequence][
            "counts_before_collapse"
        ].sum()
    sums.append(total_collapsed_counts)
df = pd.DataFrame(
    zip(most_abundant_sequence, sums),
    columns=["most_abundant_sequence", "total_collapsed_counts"],
)

merged_df = pd.merge(
    dict_df, df, how="left", left_on="sequence", right_on="most_abundant_sequence"
)
merged_df["total_collapsed_counts"].fillna(0, inplace=True)
merged_df.drop("most_abundant_sequence", axis=1, inplace=True)
merged_df.sort_values(by="counts_before_collapse", inplace=True)

ed_highest = []
ed_lowest = []
non_zero_bc = merged_df[merged_df["total_collapsed_counts"] > 0]

for upper_barcode in merged_df.iloc[(len(merged_df) - 100) :]["sequence"]:
    for lower_barcode in merged_df.iloc[: (len(merged_df) - 100)]["sequence"]:
        ed_highest.append(editdistance.eval(upper_barcode, lower_barcode))

for lower_barcode in non_zero_bc.iloc[:100]["sequence"]:
    for upper_barcode in non_zero_bc.iloc[100:]["sequence"]:
        ed_lowest.append(editdistance.eval(upper_barcode, lower_barcode))

In [ ]:
plt.title("Edit distribution post collapse for highest and lowest abundance barcodes")
plt.hist(ed_highest, bins=10, label="Top 100 barcodes", density=True)
plt.hist(ed_lowest, bins=10, alpha=0.6, label="Bottom 100 barcodes", density=True)
plt.xlabel("Edit distance")
plt.legend()

In [ ]:
# Plot barcode counts pre and post collapse
plt.title("Edit distance 2 collapse (adjacency)")
plt.plot(
    merged_df["counts_before_collapse"].values, label="Pre collapsed barcode counts"
)
plt.plot(
    merged_df["total_collapsed_counts"].values,
    alpha=0.5,
    label="Post collapsed barcode counts",
)
plt.legend()
plt.xlabel("Barcode index")
plt.ylabel("Barcode count")
# limit to see collapse to zero for many less abundant barcodes
plt.ylim(0, 30)

In [ ]:
# Load R2 library data and find in situ sequenced bases
r2_sequences = pd.read_csv(
    "/nemo/lab/znamenskiyp/home/shared/projects/barcode_diversity_analysis/collapsed_barcodes/R2/R2_bowtie_ed2.txt",
    delimiter="\t",
    header=None,
    names=["counts", "sequences"],
)
r2_sequences["10bp_seq"] = r2_sequences["sequences"].str[-10:]

In [ ]:
# Calculate edit distances between in situ barcodes and R2 library barcodes
# Levenshtein distance - not used for final plots

from multiprocessing import Pool
from tqdm import tqdm


lib_10bp_seq = np.array(r2_sequences["10bp_seq"])


# Define a function to calculate the minimum edit distance
def calculate_min_edit_distance(insitu_bc):
    edit_distances = np.fromiter(
        (editdistance.eval(insitu_bc, lib_bc) for lib_bc in lib_10bp_seq), int
    )
    min_edit_distance_idx = np.argmin(edit_distances)
    min_edit_distance = edit_distances[min_edit_distance_idx]
    lib_bc_sequence = r2_sequences.loc[min_edit_distance_idx, "10bp_seq"]
    lib_bc_count = r2_sequences.loc[min_edit_distance_idx, "counts"]
    return min_edit_distance, lib_bc_sequence, lib_bc_count


# Wrap the outer loop with tqdm for progress tracking
with Pool() as pool:
    results = list(
        tqdm(
            pool.imap(calculate_min_edit_distance, merged_df["sequence"]),
            total=len(merged_df),
            desc="Calculating edit distances",
        )
    )

# Extract the results from the list of tuples
min_edit_distances, lib_bc_sequences, lib_bc_counts = zip(*results)

# Assign the minimum edit distances, lib_bc sequences, and counts to new columns in merged_df
merged_df["min_edit_distance"] = min_edit_distances
merged_df["lib_bc_sequence"] = lib_bc_sequences
merged_df["lib_bc_counts"] = lib_bc_counts

In [ ]:
# Calculate edit distances between in situ barcodes and R2 library barcodes
# Hamming distance function - used for final plots

from multiprocessing import Pool
from tqdm import tqdm


def hamming_distance(str1, str2):
    return sum(c1 != c2 for c1, c2 in zip(str1, str2))


lib_10bp_seq = np.array(r2_sequences["10bp_seq"])


# Define a function to calculate the minimum edit distance
def calculate_min_edit_distance(insitu_bc):
    edit_distances = np.fromiter(
        (hamming_distance(insitu_bc, lib_bc) for lib_bc in lib_10bp_seq), int
    )
    min_edit_distance_idx = np.argmin(edit_distances)
    min_edit_distance = edit_distances[min_edit_distance_idx]
    lib_bc_sequence = r2_sequences.loc[min_edit_distance_idx, "10bp_seq"]
    lib_bc_count = r2_sequences.loc[min_edit_distance_idx, "counts"]
    return min_edit_distance, lib_bc_sequence, lib_bc_count


# Wrap the outer loop with tqdm for progress tracking
with Pool() as pool:
    results = list(
        tqdm(
            pool.imap(calculate_min_edit_distance, merged_df["sequence"]),
            total=len(merged_df),
            desc="Calculating edit distances",
        )
    )

# Extract the results from the list of tuples
min_edit_distances, lib_bc_sequences, lib_bc_counts = zip(*results)

# Assign the minimum edit distances, lib_bc sequences, and counts to new columns in merged_df
merged_df["ham_min_edit_distance"] = min_edit_distances
merged_df["ham_lib_bc_sequence"] = lib_bc_sequences
merged_df["ham_lib_bc_counts"] = lib_bc_counts

In [ ]:
# Generate large number of random sequences and calculate edit distances - long processing time, use lots of cores
# Is there an analytical way to calculate the expected edit distance distribution?

redo = False
if redo:
    # Generate random DNA sequences
    num_sequences = 100000
    sequence_length = 10
    random_seed = 42  # add some meaning
    np.random.seed(random_seed)
    random_sequences = np.array(
        [
            "".join(np.random.choice(["A", "C", "G", "T"], size=sequence_length))
            for _ in range(num_sequences)
        ],
        dtype=object,
    )

    # Wrap the outer loop with tqdm for progress tracking
    with Pool() as pool:
        results = list(
            tqdm(
                pool.imap(calculate_min_edit_distance, random_sequences),
                total=len(random_sequences),
                desc="Calculating edit distances",
            )
        )

    # Extract the results from the list of tuples
    min_edit_distances, lib_bc_sequences, lib_bc_counts = zip(*results)

    random_df = pd.DataFrame(random_sequences, columns=["random_sequences"])
    # Assign the minimum edit distances, lib_bc sequences, and counts to new columns in merged_df
    random_df["min_edit_distance"] = min_edit_distances
    random_df["lib_bc_sequence"] = lib_bc_sequences
    random_df["lib_bc_counts"] = lib_bc_counts
    random_df.to_pickle(
        "/nemo/lab/znamenskiyp/home/users/becalia/data/BRYC65.1d/random_100000_barcodes.pkl"
    )
else:
    random_df = pd.read_pickle(
        "/nemo/lab/znamenskiyp/home/users/becalia/data/BRYC65.1d/random_100000_barcodes.pkl"
    )

In [ ]:
low_collapsed_reads = merged_df[merged_df["total_collapsed_counts"].between(1, 50)]
high_collapsed_reads = merged_df[merged_df["total_collapsed_counts"] > 50]
low_perfect_match = low_collapsed_reads[low_collapsed_reads["min_edit_distance"] == 0]
high_perfect_match = high_collapsed_reads[
    high_collapsed_reads["min_edit_distance"] == 0
]
random_perfect_match = random_df[random_df["min_edit_distance"] == 0]
merged_perfect_match = merged_df[merged_df["min_edit_distance"] == 0]

In [ ]:
# Plot edit distance distribution histos
bins = np.arange(5) - 0.5
plt.hist(
    random_df["min_edit_distance"],
    bins,
    range=(0, 3),
    density=True,
    histtype="step",
    label="Random sequences",
    linewidth=2,
)
plt.hist(
    high_collapsed_reads["min_edit_distance"],
    bins,
    range=(0, 3),
    density=True,
    histtype="step",
    label="In situ sequences with more than 50 spots",
    linewidth=2,
    alpha=0.7,
)
plt.xticks(range(4))
plt.xlim([-1, 4])
plt.ylim(0, 1)
plt.xlabel("Minimum edit distance to library")
plt.ylabel("Proportion of total sequences")
plt.legend()
plt.savefig(
    "/nemo/lab/znamenskiyp/home/users/becalia/figs/2023_UCL_sympo/edit_distance_to_library.svg",
    bbox_inches="tight",
)

In [ ]:
# Plot barcode abundance boxplots for perfect matches to library
fig, (ax0, ax1, ax2, ax3) = plt.subplots(
    4,
    1,
    sharex=True,
    figsize=(8, 6),
    gridspec_kw={"height_ratios": [0.08, 0.08, 0.08, 0.6]},
)

ax0.boxplot(
    high_perfect_match["lib_bc_counts"],
    vert=False,
    widths=0.7,
    boxprops=dict(color="#2ca02c"),
    whiskerprops=dict(color="#2ca02c"),
    flierprops=dict(marker="o", markersize=3, alpha=0.1, color="#2ca02c"),
    capprops=dict(color="#2ca02c"),
    medianprops=dict(color="red"),
)

# ax0.axis('off')

ax1.boxplot(
    low_perfect_match["lib_bc_counts"],
    vert=False,
    widths=0.7,
    boxprops=dict(color="#ff7f0e"),
    whiskerprops=dict(color="#ff7f0e"),
    flierprops=dict(marker="o", markersize=3, alpha=0.1, color="#ff7f0e"),
    capprops=dict(color="#ff7f0e"),
    medianprops=dict(color="red"),
)
# ax1.axis('off')
ax2.boxplot(
    random_perfect_match["lib_bc_counts"],
    vert=False,
    widths=0.7,
    boxprops=dict(color="#1f77b4"),
    whiskerprops=dict(color="#1f77b4"),
    flierprops=dict(marker="o", markersize=3, alpha=0.1, color="#1f77b4"),
    capprops=dict(color="#1f77b4"),
    medianprops=dict(color="red"),
)
# ax2.axis('off')

# Adjust spacing between subplots
plt.subplots_adjust(hspace=0.3)


sequences = np.flip(r2_sequences["counts"])
bin_edges = np.logspace(0, 4, num=80)  # number of bins
edge_positions = sequences.searchsorted(bin_edges)
counts = np.zeros(len(bin_edges) - 1)

for i in range(len(bin_edges) - 1):
    parts = edge_positions[i : i + 2]
    counts[i] = sequences[parts[0] : parts[1]].sum()
counts /= np.sum(sequences)

# Plot PDF histogram on the second subplot
ax3.plot(bin_edges[:-1], counts, drawstyle="steps-post", linewidth=0)
ax3.fill_between(bin_edges[:-1], 0, counts, step="post", alpha=0.6, color="k")
ax3.set_xlabel("Barcode abundance")
ax3.set_ylabel(r"% of total sequences")
ax3.set_xscale("log")
ax3.xaxis.set_major_locator(mticker.FixedLocator(locs=np.logspace(0, 4, 5)))
ax3.xaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))
for label in ax3.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False)
yticks = [0, 0.0125, 0.025, 0.0375, 0.05]
ax3.set_yticks(ticks=yticks)
ax3.set_yticklabels(labels=[0.00, 1.25, 2.50, 3.75, 5.00])
ax3.set_xlim(None, 10000)
ax3.set_ylim(-0.005, 0.05)
ax3.yaxis.grid(False)

plt.subplots_adjust(hspace=0.3)


def despine(ax):
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)


for ax in (ax0, ax1, ax2, ax3):
    despine(ax)

plt.savefig(
    "/nemo/lab/znamenskiyp/home/users/becalia/figs/2023_UCL_sympo/matched_library_abundance.svg",
    bbox_inches="tight",
)
plt.show()

In [ ]:
def all_pairwise_distances(df):
    allseq = np.vstack(df.sequence.values)
    all_dists = np.sum((allseq[:, np.newaxis] - allseq[np.newaxis, :, :]) != 0, axis=2)
    np.fill_diagonal(np.array(all_dists, dtype=float), np.nan)
    return all_dists


good_roi_barcodes = all_roi_barcodes[all_roi_barcodes["dot_product_score"] > 0.2]

all_dists = all_pairwise_distances(good_roi_barcodes)

# intra cell distance
print("Doing intra cells pairwise distances")
intra_dsts = dict()
for gp, df in good_roi_barcodes.groupby("mask_id"):
    intra_dsts[gp] = all_pairwise_distances(df)
in_cells = np.hstack([np.reshape(ar, -1) for ar in intra_dsts.values()])

In [ ]:
# Create plot of all barcode pairwise distances and within cell distances
fig, ax = plt.subplots(1, 1)
plt.rcParams["svg.fonttype"] = "none"

ax.hist(
    all_dists[~np.isnan(all_dists)],
    bins=np.arange(-0.5, 11),
    density=True,
    histtype="step",
    label="All pairwise distances",
    color="red",
    linewidth=2,
)

ax.hist(
    in_cells[~np.isnan(in_cells)],
    bins=np.arange(-0.5, 11),
    density=True,
    histtype="step",
    label="Within cell distances",
    color="black",
    linewidth=2,
)

ax.legend(loc="upper right")
ax.set_xlabel("Edit distance")
ax.set_ylabel("Proportion of rolonies (density)")
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.xticks(range(11))
plt.ylim(0, 0.5)
plt.savefig(
    "/nemo/lab/znamenskiyp/home/users/becalia/figs/2023_UCL_sympo/all_barcode_0.2_ed.svg",
    bbox_inches="tight",
)

In [ ]:
# Create heatmap plot of edit distances within each cell
distance_df = distance_df.sort_values(0)
fig, ax = plt.subplots(1, 1, figsize=(3, 6))
im = ax.imshow(
    distance_df.iloc[:2057].values, aspect="auto", interpolation="None", vmax=50
)
cb = fig.colorbar(im, ax=ax)
cb.set_label("# rolonies")
ax.set_xlabel("Edit distance")
ax.set_xticks(range(11))
ax.set_ylabel("Cell #")
plt.savefig(
    "/nemo/lab/znamenskiyp/home/users/becalia/figs/2023_UCL_sympo/within_cell_ed_heatmap.svg",
    bbox_inches="tight",
)